In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.data.experimental import TFRecordWriter
from sklearn.datasets import load_sample_image
from IPython.display import Image
from tensorflow.train import BytesList, FloatList, Int64List
from tensorflow.train import Example, Features, Feature
import tfrecord
import cv2
import glob
import torchvision
import os
import pandas as pd
from torchvision.io import read_image
import torch
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt
import torch
import copy
import torch.nn as nn
import gc

In [2]:
print(torch.cuda.is_available())
print(torch.backends.cudnn.version())
print(torch.backends.cudnn.m.is_available())
print(torch.backends)
# print(torch.ones(5,5, device="opencl"))

True
8200
True
<module 'torch.backends' from 'C:\\Users\\alex\\anaconda3\\lib\\site-packages\\torch\\backends\\__init__.py'>


In [3]:
import torch
import numpy as np
import time

In [4]:
a = torch.randint(1, 1000, (10000 * 5, 10000))
print(a)
print("Device Name: ", a.device)

tensor([[918, 199, 296,  ...,  10, 857, 163],
        [447, 323,  54,  ..., 859, 492, 490],
        [152, 391, 330,  ...,   3, 121, 345],
        ...,
        [  6, 579, 643,  ..., 284, 708, 610],
        [655, 195, 357,  ..., 851,  63, 680],
        [287, 605, 933,  ..., 220, 693, 940]])
Device Name:  cpu


In [5]:
aa = (((((a ** (1/22)) ** 3) ** 0.1) * 2) ** 3)
print(aa)

tensor([[10.5754,  9.9342, 10.0969,  ...,  8.7902, 10.5457,  9.8535],
        [10.2686, 10.1330,  9.4180,  ..., 10.5467, 10.3090, 10.3073],
        [ 9.8253, 10.2125, 10.1419,  ...,  8.3677,  9.7341, 10.1604],
        ...,
        [ 8.6084, 10.3779, 10.4225,  ..., 10.0798, 10.4636, 10.4000],
        [10.4304,  9.9260, 10.1746,  ..., 10.5427,  9.4776, 10.4464],
        [10.0842, 10.3965, 10.5824,  ...,  9.9751, 10.4545, 10.5857]])


In [6]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
cuda_id = torch.cuda.current_device()
print("CUDA Device ID: ", torch.cuda.current_device())
print("Name of the current CUDA Device: ", torch.cuda.get_device_name(cuda_id))
torch.cuda.set_device(0)

cuda
CUDA Device ID:  0
Name of the current CUDA Device:  NVIDIA GeForce RTX 3060


In [7]:
# gc.collect()
b = torch.randint(1, 1000, (10000 * 5, 10000))
b = b.to(torch.device("cuda"))
print(b)
print("Device Name: ", b.device)
# torch.cuda.empty_cache()

tensor([[930, 462, 807,  ..., 311, 246, 306],
        [203, 825, 947,  ...,  94, 917,  58],
        [ 30, 502, 682,  ...,  23, 689, 479],
        ...,
        [187, 764, 852,  ..., 388, 323, 218],
        [186, 886, 425,  ..., 825,  21, 354],
        [752, 144, 586,  ..., 838, 773, 538]], device='cuda:0')
Device Name:  cuda:0


In [8]:
# torch.cuda.empty_cache()

In [9]:
bb = (((((b ** (1/22)) ** 3) ** 0.1) * 2) ** 3)
print(bb)

tensor([[10.5810, 10.2825, 10.5198,  ..., 10.1173, 10.0208, 10.1106],
        [ 9.9423, 10.5293, 10.5889,  ...,  9.6341, 10.5749,  9.4456],
        [ 9.1943, 10.3175, 10.4476,  ...,  9.0949, 10.4520, 10.2977],
        ...,
        [ 9.9090, 10.4963, 10.5432,  ..., 10.2093, 10.1330,  9.9714],
        [ 9.9068, 10.5601, 10.2474,  ..., 10.5293,  9.0611, 10.1711],
        [10.4895,  9.8036, 10.3830,  ..., 10.5360, 10.5013, 10.3467]],
       device='cuda:0')


In [16]:
## CPU ONLY
start_time = time.time()
aa = (((((a ** (1/22)) ** 3) ** 0.1) * 2) ** 3)
print(aa)
print("--- device %s, time: %s ---" % (a.device, time.time() - start_time))

tensor([[10.5754,  9.9342, 10.0969,  ...,  8.7902, 10.5457,  9.8535],
        [10.2686, 10.1330,  9.4180,  ..., 10.5467, 10.3090, 10.3073],
        [ 9.8253, 10.2125, 10.1419,  ...,  8.3677,  9.7341, 10.1604],
        ...,
        [ 8.6084, 10.3779, 10.4225,  ..., 10.0798, 10.4636, 10.4000],
        [10.4304,  9.9260, 10.1746,  ..., 10.5427,  9.4776, 10.4464],
        [10.0842, 10.3965, 10.5824,  ...,  9.9751, 10.4545, 10.5857]])
--- device cpu, time: 3.840715169906616 ---


In [17]:
## GPU
start_time = time.time()
bb = (((((b ** (1/22)) ** 3) ** 0.1) * 2) ** 3)
print(bb)
print("--- device %s, time: %s ---" % (b.device, time.time() - start_time))

tensor([[10.5810, 10.2825, 10.5198,  ..., 10.1173, 10.0208, 10.1106],
        [ 9.9423, 10.5293, 10.5889,  ...,  9.6341, 10.5749,  9.4456],
        [ 9.1943, 10.3175, 10.4476,  ...,  9.0949, 10.4520, 10.2977],
        ...,
        [ 9.9090, 10.4963, 10.5432,  ..., 10.2093, 10.1330,  9.9714],
        [ 9.9068, 10.5601, 10.2474,  ..., 10.5293,  9.0611, 10.1711],
        [10.4895,  9.8036, 10.3830,  ..., 10.5360, 10.5013, 10.3467]],
       device='cuda:0')
--- device cuda:0, time: 0.3745567798614502 ---
